In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from PIL import Image

import tensorflow as tf

import os
import shutil
import pickle

%matplotlib inline

In [ ]:
# Create the temporary directories.
if not os.path.exists('temp_folder'):
    os.makedirs('temp_folder')

if not os.path.exists('temp_folder/non_nv/'):
    os.makedirs('temp_folder/non_nv/')

In [ ]:
BATCH_SIZE = 16
SEED = 10

IM_HEIGHT_NV = 168
IM_WIDTH_NV = 224

IM_HEIGHT_REST = 225
IM_WIDTH_REST = 300

np.random.seed(SEED)

global_mean_nv = np.load('../../Data/Skin/nv_non_nv/mean_image_nv_non_nv.npz')['image'].astype(np.uint8)
nv_mean_image = (Image.fromarray(global_mean_nv)).resize((IM_WIDTH_NV, IM_HEIGHT_NV))
global_mean_nv = np.asarray(nv_mean_image).astype(np.float32)

global_mean_rest = np.load('../../Data/Skin/six_way_level_two/mean_image_six_way_level_two.npz')['image'].astype(np.uint8)
rest_mean_image = (Image.fromarray(global_mean_rest)).resize((IM_WIDTH_REST, IM_HEIGHT_REST))
global_mean_rest = np.asarray(rest_mean_image).astype(np.float32)


model_name_nv = 'nv_non_nv_inceptionv3_512_custom_weight_init_full_train-ve1.h5'
model_path_nv = '/home/ubuntu/Notebooks/Rahul/Hierarchical/Finalized Hierarchical Models/' + model_name_nv

model_name_rest = 'six_way_level_two_inceptionv3_256_full_train_gauss_noise.h5'
model_path_rest = '/home/ubuntu/Notebooks/Bhushan/' + model_name_rest

print(global_mean_nv.shape)
print(global_mean_rest.shape)

In [ ]:
def preprocess_nv_image(image):
    return (image - global_mean_nv)

def preprocess_rest_image(image):
    return (image - global_mean_rest)

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return (true_positives / (predicted_positives + K.epsilon()))

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return (true_positives / (possible_positives + K.epsilon()))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight
from PIL import Image
from keras.models import load_model

In [ ]:
img_gen_nv = ImageDataGenerator(preprocessing_function=preprocess_nv_image, rescale=1./255.)
# img_gen_rest = ImageDataGenerator(preprocessing_function=preprocess_rest_image, rescale=1./255.)

val_gen_nv = img_gen_nv.flow_from_directory('/home/ubuntu/Data/Skin/Test/', 
                                        target_size=(IM_HEIGHT_NV, IM_WIDTH_NV), 
                                        class_mode=None, 
                                        shuffle=False, 
                                        batch_size=BATCH_SIZE)

# Load NV - Non-NV model
level_one_model = load_model(model_path_nv, compile=True, custom_objects={'f1': f1,
                                                                          'precision': precision,
                                                                          'recall': recall})

level_one_preds = level_one_model.predict_generator(val_gen_nv,
                                                    verbose=1)
print(level_one_preds.shape)

In [ ]:
# NV-Non_NV
val_files = val_gen_nv.filenames

nv_indices = np.where(level_one_preds > 0.5)[0]
non_nv_indices = np.where(level_one_preds <= 0.5)[0]

nv_preds = level_one_preds[level_one_preds > 0.5]
non_nv_preds = level_one_preds[level_one_preds <= 0.5]

nv_files = [val_files[x] for x in nv_indices]
non_nv_files = [val_files[x] for x in non_nv_indices]

nv_dict = dict({nv_files[i]: nv_preds[i] for i in range(len(nv_files))})

with open('nv_predictions_test.pickle', 'wb') as handle:
    pickle.dump(nv_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print('Dumped NV records\nImages identified as NV: {}'.format(len(nv_dict)))

In [ ]:
# Save the non-nv files.
for non_nv_file in non_nv_files:
    shutil.copy(src='/home/ubuntu/Data/Skin/Test/' + non_nv_file,
               dst='temp_folder/non_nv/' + non_nv_file[4:])

print('Done copying to temporary folder')

# Done with NV

In [ ]:
# img_gen_nv = ImageDataGenerator(preprocessing_function=preprocess_nv_image, rescale=1./255.)
img_gen_rest = ImageDataGenerator(preprocessing_function=preprocess_rest_image, rescale=1./255.)

val_gen_rest = img_gen_rest.flow_from_directory('temp_folder/', 
                                        target_size=(IM_HEIGHT_REST, IM_WIDTH_REST), 
                                        class_mode=None, 
                                        shuffle=False, 
                                        batch_size=BATCH_SIZE)

# Load 6 way model model
level_two_model = load_model(model_path_rest, compile=True, custom_objects={'f1': f1,
                                                                          'precision': precision,
                                                                          'recall': recall})

level_two_preds = level_two_model.predict_generator(val_gen_rest, verbose=1)
print(level_two_preds.shape)


In [ ]:
rest_files = val_gen_rest.filenames

predictions = np.zeros((1512, 7))

with open('nv_predictions_test.pickle', 'rb') as handle:
    nv_predictions = pickle.load(handle)
# print(nv_predictions)
filenames = []
idx = 0

for filename in nv_predictions.keys():
    img_name = filename[filename.index('/') + 1:]
    filenames.append(img_name)
    predictions[idx, 1] = nv_predictions[filename]
    idx += 1

n_nv = idx
for file in rest_files:
    filename = file[7:]
    filenames.append(filename)
    predictions[idx, 0] = level_two_preds[idx - n_nv, 0]
    predictions[idx, 1] = 0
    predictions[idx, 2:] = level_two_preds[idx - n_nv, 1:]
    idx += 1
    
print(idx)
np.savez_compressed('pred_test.npz', pred = predictions)
print('Saved MoFo')

In [ ]:
import pandas as pd
csv_frame = pd.DataFrame()
# map_indices_rev = {val: key for key, val in map_indices.items()}
csv_frame['image'] = [filename.strip('.jpg') for filename in filenames]
class_list = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
for idx in range(7):
    csv_frame[class_list[idx]] = predictions[:, idx]
    
csv_frame

In [ ]:
csv_frame = csv_frame.sort_values('image')
csv_frame.to_csv('test_sixway_predictions.csv', index=False)
# csv_frame.head(5)